### Install

In [ ]:
# run this cell to install Julia
# F5 to refresh

%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia"
JULIA_NUM_THREADS=9
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key)"
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


### Julia

In [ ]:
using Pkg

# adding libarries
Pkg.add("LinearAlgebra")
Pkg.add("JLD")
Pkg.add("SparseArrays")

# including libraries
using LinearAlgebra
using JLD
using SparseArrays

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


### Problem 1

In [ ]:
# function construct matrix G from probability matrix H and alpha
function construct_G(H, alpha)
  m,n = size(H)

  # dangling vector
  a = zeros(m)
  for i=1:m
    a[i] = 1-sum(H[i,:])
  end

  # stochastic vector u
  u = ones(m)/m

  # stochastic matrix B
  B = H + a*u'

  # vector e
  e = ones(m)

  # primitive matrix H
  G = alpha*B + (1-alpha)*e*u'
  return G
end

# function power method, inputs are matrix G, number of iterations, and small real positive number epsilon
function power_method(G, iterations, epsilon)
  m,n = size(G)

  # counting number of actual iterations
  count = iterations

  # initial stochastic vector v
  v = ones(m)/m

  # compute eigenvector of G
  for i=1:iterations
    v_old = copy(v)
    v = G'*v

    # stop the loop if 2-norm of (v-v_old) less than epsilon
    if(norm(v-v_old) < epsilon)
      count = i
      break
    end
  end

  # return vector v
  return v, count
end

# probability matrix
H = [0   1/2  1/2  0    0;
     0   0    0    1    0;
     1/3 1/3  0    1/3  0;
     1/3 0    1/3  0    1/3;
     1   0    0    0    0]

# Construct matrix G from H and alpha
alpha = 0.8
G = construct_G(H, alpha)

# apply power method to matrix G
iterations = 100
epsilon = 1e-10
v,count = power_method(G, iterations, epsilon)

# PageRank vector
rank = reverse(sortperm(v))

# print them
println("-- number of iterations: ", count)
println("-- v: ", v)
println("-- rank: ", rank)

-- number of iterations: 35
-- v: [0.24626064553569044, 0.1931582337497339, 0.20495240821732383, 0.24918056250356288, 0.10644814999368796]
-- rank: [4, 1, 3, 2, 5]


### Problem 2

In [ ]:
# function power method for sparse matrix H
function power_method_sparse(H, a, iterations, alpha, epsilon)
  m,n = size(H)

  # counting number of actual iterations
  count = iterations

  # initial vector v, u, e, alpha
  v = [[1]; zeros(m-1,1)]
  u = ones(m,1)/m
  e = ones(m,1)

  # compute eigenvector of H
  for i=1:iterations
    v_old = copy(v)
    v = alpha*H'*v + alpha*u*(v'*a) + (1-alpha)*u*(v'*e)

    # stop the loop if 2-norm of (v-v_old) less than epsilon
    if(norm(v-v_old) < epsilon)
      count = i
      break
    end
  end

  # return vector v and number of iterations
  return v, count
end

# load data from JLD file
data = load("/content/stanford.jld")
H = data["S"]
a = data["a"]

# apply power method to matrix H
iterations = 100
alpha = 0.6
epsilon = 1e-3
v, count = power_method_sparse(H, a, iterations, alpha, epsilon)

# PageRank vector
rank = reverse(sortperm(v[:,1]))

# print them
println("-- number of iterations: ", count)
println("-- 10 highest ranking pages: ", rank[1:10])

-- number of iterations: 14
-- 10 highest ranking pages: [226411, 89073, 241454, 67756, 69358, 134832, 225872, 186750, 231363, 234704]


### Problem 3

In [ ]:
# function find eigenvalues of matrix A with number of iterations and small positive real number epsilon
function find_eigvals(A, iterations, epsilon)

  count = 1
  # loop for computing matrix A_{k+1} from A_k
  for i=1:iterations
    A_old = copy(A)
    Q,R = qr(A)
    A = R*Q

    # if 2-norm of (A_{k+1} - A_k) less than epsilon then stop
    if(norm(A-A_old) < epsilon)
      count = i
      break
    end
  end

  # return matrix A
  return A, count
end

# matrix A
A = [9  -7  -6  -13 -5;
    -7   5  -14 -8  -9;
    -6  -14  5  -5  -1;
    -13 -8  -5  13   5;
    -5  -9  -1  5   13]

# apply function find_eigvals to matrix A
iterations = 1e5
epsilon = 1e-6
A_k, count = find_eigvals(A, iterations, epsilon)

# print them
display(A_k)
println("\n--- number of iterations: ", count)
println("\nChecking:\n--- eigenvalues of A (by QR algorithm): \n", sort(diag(A_k)))
println("\n--- eigenvalues of A (by eigvalues function): \n", sort(eigvals(A)))

5×5 Matrix{Float64}:
 29.5563          9.87254e-15   -2.05257e-16    5.73448e-15    2.57031e-15
 -5.27134e-176  -20.1881         3.49255e-7     7.12413e-15   -2.07205e-15
  4.81879e-184    3.49255e-7    19.8148         1.33206e-14    4.74111e-15
  0.0            -1.39784e-308   6.59806e-301  10.3734         3.36138e-15
  0.0             0.0            0.0           -2.35063e-299   5.44368


--- number of iterations: 1069.0

Checking:
--- eigenvalues of A (by QR algorithm): 
[-20.1881352648135, 5.44367704938099, 10.373436381368819, 19.81475666930468, 29.556265164759363]

--- eigenvalues of A (by eigvalues function): 
[-20.188135264813763, 5.4436770493809945, 10.373436381368807, 19.814756669304543, 29.556265164759413]


I printed the number of iterations I used above.

I'll stop the loop by condition: 2-norm(A_{k+1} - A_k) less than a given epsilon

### Problem 4

In [ ]:
# -------------------------
# ---------- (a) ----------
# -------------------------
# create a function find_docs
function find_docs(A, terms, docs, query, alpha)
  # normalize A
  m,n = size(A)
  for i=1:n
    A[:,i] = A[:,i]/norm(A[:,i])
  end

  # create vector query q
  w = split(query)
  q = zeros(m,1)
  for i=1:size(w)[1]
    indx = findall(x -> x==w[i], terms)
    q[indx[1]] = 1
  end

  # normalize q
  q = q/(norm(q))

  # compute w
  w = A'*q

  # decide what doc is relevant
  result = zeros(n,1)
  for i=1:n
    if(w[i] >= alpha)
      result[i] = 1
    end
  end

  # create relevant_docs vector (vector of strings)
  relevant_docs = String[]
  indx = findall(x -> x==1, result)
  for i=1:size(indx)[1]
    push!(relevant_docs, docs[indx[i]])
  end

  # return relevant docs
  return relevant_docs
end

# load data from JLD file
data = load("/content/shake.jld")
A = data["tdm"]
terms = data["dictionary"]
docs = data["plays"]

# print relevant docs
query = "wherefore art thou"
alpha = 0.3
rel_docs = find_docs(A, terms, docs, query, alpha)
println("(a): ", rel_docs)


# -------------------------
# ---------- (b) ----------
# -------------------------
query = "lady doth protest much"
alpha = 0.1
rel_docs = find_docs(A, terms, docs, query, alpha)
println("(b): ", rel_docs)


# -------------------------
# ---------- (c) ----------
# -------------------------
# create a function to find low rank approximation matrix
function low_rank_approx(A, k)
  # create U_k, S_k, V_k
  U,S,V = svd(A)
  U_k = U[:,1:k]
  S_k = S[1:k]
  V_k = V[:,1:k]

  # construct approximated matrix A_k
  A_k = U_k*diagm(S_k)*V_k'

  # return matrix A_k
  return A_k
end

# construct A_k
k = 10
A_k = low_rank_approx(A, k)

# part (c_a)
query = "wherefore art thou"
alpha = 0.3
rel_docs = find_docs(A_k, terms, docs, query, alpha)
println("(c_a): ", rel_docs)

# part (c_b)
query = "lady doth protest much"
alpha = 0.1
rel_docs = find_docs(A_k, terms, docs, query, alpha)
println("(c_b): ", rel_docs)

(a): ["The Tempest", "Henry IV, part 1", "Romeo and Juliet"]
(b): ["Love's Labours Lost", "The Merchant of Venice", "Much Ado About Nothing", "Twelfth Night", "Romeo and Juliet"]
(c_a): ["The Tempest", "Henry IV, part 1", "Romeo and Juliet", "Timon of Athens"]
(c_b): ["Love's Labours Lost", "The Merchant of Venice", "A Midsummer Night's Dream", "Much Ado About Nothing", "Twelfth Night", "Two Gentlemen of Verona"]


### Problem 5

I'll create matrix H by Julia code, but at first I want to have a correct matrix to test

In [ ]:
test = [0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0;
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36;
1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18;
1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12;
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6 5/36;
5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36 1/6;
1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9 5/36;
5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12 1/9;
1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18 1/12;
1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36 1/18;
1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/36;
1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
0 1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

40×40 Matrix{Float64}:
 0.0        0.0        0.0277778  …  0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0        …  0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0        …  0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0
 ⋮                                ⋱                        
 0.0277778  0.0        0.0           0.111111   0.0833333  0.0555556
 0.0555556  0.0277778  0.0   

In [ ]:
# function return H^k
function H_power_k(H, k)
  K = copy(H)
  for i=2:k
    K = K*H
  end
  return K
end

# choose a very big number (just for checking) to check if H is primitive or not
count = 0
for i=1:1000
  if(minimum(H_power_k(test,i)) > 0)
    count = i
    print("H^k positive")
    break
  end
end

if(count > 0)
  print(" when k = ", count, ", so H is primitive")
else
  print("Maybe H is not primitive")
end

H^k positive when k = 5, so H is primitive

I don't want to break the game rule, so I apply power method directly to matrix H (I checked that H is primitive)

In [ ]:
# create matrix H
p = [1/36 1/18 1/12 1/9 5/36 1/6 5/36 1/9 1/12 1/18 1/36]
H = zeros(40,40)
for i=1:40
  for j=1:11
    z = 1+i+j
    if(z > 40)
      H[i, z-40] = p[j]
    else
      H[i, z] = p[j]
    end
  end
end
H[31,:] = zeros(1,40)
H[31,11] = 1

# test
if(norm(H - test) == 0)
  println("Create matrix H successfully")
end

# apply power method to matrix H
iterations = 1000
epsilon = 1e-10
v,count = power_method(H, iterations, epsilon)

# Rank vector
rank = reverse(sortperm(v))

# print them
println("-- number of iterations: ", count)
println("-- v: ", v)
println("-- rank: ", rank)

Create matrix H successfully
-- number of iterations: 166
-- v: [0.02233950006905977, 0.022513293930103224, 0.022703227390427426, 0.02295522773158045, 0.022593416524825896, 0.022403849805061826, 0.022334436349703092, 0.022345033599389393, 0.022402284146736264, 0.022475263703183368, 0.04869197460760772, 0.022532715943665467, 0.02323857035644638, 0.02394060349477708, 0.02466789212559549, 0.025444176040476628, 0.026291753044788136, 0.027235132170195776, 0.02683132297321909, 0.026554556833927864, 0.026352474544575115, 0.0261786806838549, 0.025988747223876018, 0.025736746883135334, 0.02609855808974115, 0.026288124809512536, 0.0263575382650128, 0.026346941015571234, 0.026289690468534823, 0.026216710912422433, 0.02616050771882802, 0.026159258672509768, 0.025453404259246956, 0.024751371120377987, 0.0240240824889631, 0.023247798573425613, 0.02240022156839597, 0.021456842442206296, 0.021860651639671963, 0.022137417779363757]
-- rank: [11, 18, 19, 20, 27, 21, 28, 17, 29, 26, 30, 22, 31, 32, 25, 2